# Organize RGC
---
*8/19/2022 Gregory Matousek*

This program will read in the .root files from SIDISKinematicsReco and organize them in terms of target and HWP. This simply comprises of moving the files to a new location in /volatile/

In [1]:
import ROOT
import sys
import os
from pathlib import Path
sys.path.append("/work/clas12/users/gmat/packages/clas12root/rcdb/python/")

import rcdb
from rcdb.provider import RCDBProvider
from rcdb.model import ConditionType

import pandas as pd

Welcome to JupyROOT 6.22/06


---
## Input

Directory containing analyzed .root files

In [2]:
filedir = "/volatile/clas12/users/gmat/tutorial_2c/test_2c/"
files = []
for path in Path(filedir).glob("*.*"):
    files.append(str(path))

## End of Input
---
Read in the CLAS RCDB

In [3]:
db = RCDBProvider("mysql://rcdb@clasdb/rcdb")

Pull RCDB information from the runs and store them in arrays for the files

In [4]:
cols = []
cnds = []
def add(str1,str2):
    cols.append(str1)
    cnds.append(str2)
add("beamE","beam_energy")
add("beamCurrent","beam_current")
add("Nevents","event_count")
add("eventRate","events_rate")
add("isValid","is_valid_run_end")
add("Tpol","target_polarization")
add("HWP","half_wave_plate")
add("Target","target")

df = pd.DataFrame(columns=["Run"]+cols)

for file in files:
    f = ROOT.TFile(file,"READ")
    if(not f.GetListOfKeys().Contains("tree_reco")):
        print("ERROR: Missing tree_reco in file",file,"...skipping...")
        continue
    t = f.Get("tree_reco")
    for iev in t:
        # Get run from TTree
        runNumber = int(iev.run)
        # Get run from database
        run = db.get_run(runNumber)
        # Setup conditions from database
        vallist=[runNumber]
        for cnd in cnds:
            cnd_val = run.get_condition(cnd).value
            if(isinstance(cnd_val,str)):
                cnd_val=cnd_val.strip()
            vallist.append(cnd_val)
        # Add list to dataframe
        df.loc[len(df)] = vallist
        break
    f.Close()
display(df)
df.to_csv(filedir+'rcdb.csv')

ERROR: Missing tree_reco in file /volatile/clas12/users/gmat/tutorial_2c/test_2c/test_2c_.root ...skipping...


,Run,beamE,beamCurrent,Nevents,eventRate,isValid,Tpol,HWP,Target
0,16769,10547.3,3.472740,26812371,129.2350,False,0.575167,1,NH3
1,16467,10547.3,4.077020,75649881,6190.4200,False,-0.272752,1,ND3
2,16758,10547.3,3.685060,56603956,104.9510,False,-0.497576,1,NH3
3,16615,10547.3,4.085410,52092621,804.2860,True,-0.205105,0,ND3
4,16771,10547.3,3.880160,66084464,15.9286,True,0.576812,1,NH3
...,...,...,...,...,...,...,...,...,...
76,16484,10547.3,3.882590,65411418,922.5790,True,-0.407541,0,ND3
77,16752,10547.3,3.969540,21167307,157.8630,True,-0.510760,0,NH3
78,16281,10547.3,2.593910,56934341,25.6364,True,0.060331,0,ND3
79,16422,10547.3,0.004414,63680501,2593.0700,True,0.384794,0,ND3


In [116]:
runNumberArr = df['Run'].to_numpy()
HWPArr = df['HWP'].to_numpy()
targetArr = df['Target'].to_numpy()

Make the relevant directories

In [117]:
targs = ['ND3','NH3','C']
hwps = ['hwp-in','hwp-out']
for _t in targs:
    for _h in hwps:
        path = filedir+_t+'-'+_h
        if(not os.path.exists(path)):
            os.mkdir(filedir+_t+'-'+_h)
        else:
            print("Exception: Directory",path,"already exists. Skipping...")

Move the analysis files into their respective subdirectories based on their simulation parameters

In [118]:
for _file,_target,_hwp,_runNumber in zip(files,targetArr,HWPArr,runNumberArr):
    # Default path
    path = filedir
    # Find target type
    if(_target=="ND3"):
        path = path + "ND3"
    elif(_target=="NH3"):
        path = path + "NH3"
    elif(_target=="C" or _target=="12C"):
        path = path + "C"
    else:
        print("ERROR: Unknown target type",_target,"in file",_file,"...skipping...")
        continue
    # Find HWP param
    if(_hwp == 0):
        path = path + "-hwp-out"
    elif(_hwp == 1):
        path = path + "-hwp-in"
    else:
        print("ERROR: Unknown HWP param",_hwp,"in file",_file,"...skipping...")
        continue
    # Create new file name
    path = path + "/run-" + str(int(_runNumber)) + ".root"
    # Move file
    current = _file
    destination = path
    if(os.path.exists(current)):
        os.rename(current,destination)
        print("Successfully moved file with run number",int(_runNumber))
    else:
        print("ERROR: File",current,"does not exist...skipping...")
        continue

Successfully moved file with run number 16423
Successfully moved file with run number 16280
Successfully moved file with run number 16432
Successfully moved file with run number 16338
Successfully moved file with run number 16289
Successfully moved file with run number 16467
Successfully moved file with run number 16285
Successfully moved file with run number 16327
Successfully moved file with run number 16405
Successfully moved file with run number 16409
Successfully moved file with run number 16628
Successfully moved file with run number 16484
Successfully moved file with run number 16335
Successfully moved file with run number 16411
Successfully moved file with run number 16617
Successfully moved file with run number 16416
Successfully moved file with run number 16629
Successfully moved file with run number 16408
Successfully moved file with run number 16336
Successfully moved file with run number 16483
Successfully moved file with run number 16410
Successfully moved file with run n